# Ziel des Notebooks
* Wir validieren, ob wir die BirdNET Python Library so zum Laufen kriegen, dass sie:

    * Exakt (oder fast exakt) die gleichen Ergebnisse liefert wie deine Birdnet-Analyzer-App.

    * Auf deiner CPU schnell genug läuft.

* Relevante Funktionen aus der Dokumentation nutzen

In [ ]:
import os, sys, platform, json, time
from pathlib import Path

print("Python:", sys.version)
print("Platform:", platform.platform())
print("CWD:", Path.cwd())

PROJECT_ROOT = Path.cwd().parent  # passe an, falls du im notebooks/ Ordner bist
DATA_DIR = Path("E:/Samsung SSD 850 EVO extern/wmv")   
OUT_DIR = PROJECT_ROOT / "outputs" / "experiments" / "birdnet"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_DIR exists:", DATA_DIR.exists(), DATA_DIR)
print("OUT_DIR:", OUT_DIR)

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
CWD: /content
DATA_DIR exists: False E:/Samsung SSD 850 EVO extern/wmv
OUT_DIR: /outputs/experiments/birdnet


# BirdNET importieren und Version testen

In [7]:
import birdnet
print("birdnet version:", getattr(birdnet, "__version__", "unknown"))


ModuleNotFoundError: No module named 'birdnet'

In [5]:
from importlib.metadata import version, PackageNotFoundError

for pkg in ["birdnet", "birdnet-analyzer", "birdnet_analyzer"]:
    try:
        print(pkg, "->", version(pkg))
    except PackageNotFoundError:
        print(pkg, "-> not installed (as distribution name)")


birdnet -> 0.2.11
birdnet-analyzer -> not installed (as distribution name)
birdnet_analyzer -> not installed (as distribution name)


In [ ]:
print("CPU cores:", os.cpu_count())

CPU cores: 2


In [10]:
wav_files = sorted(list(DATA_DIR.glob("*.WAV")))
print(f'Anzahl WAVs ist: {len(wav_files)}')

wav_path = Path("C:/Users/mghan/Desktop/BirdDetection_BA/audio_48k/2453AC0263FBD00C_20250610_042001_first30min.wav")
print(f'Beispieldatei: {wav_path}')
print(wav_path.exists())

Anzahl WAVs ist: 0
Beispieldatei: C:/Users/mghan/Desktop/BirdDetection_BA/audio_48k/2453AC0263FBD00C_20250610_042001_first30min.wav
False


In [11]:
Path.cwd()

PosixPath('/content')

# BirdNET Modell laden

In [8]:
model = birdnet.load("acoustic", "2.4", "tf")
print("Model loaded:", model)

Model loaded: <birdnet.acoustic_models.v2_4.model.AcousticModelV2_4 object at 0x000001BD9F852310>


# Predict auf eine Datei

In [9]:
import inspect
print(inspect.signature(model.predict))
print(inspect.getdoc(model.predict))

(inp: 'Path | str | Iterable[Path | str]', /, *, top_k: 'int | None' = 5, n_feeders: 'int' = 1, n_workers: 'int | None' = None, batch_size: 'int' = 1, prefetch_ratio: 'int' = 1, overlap_duration_s: 'float' = 0, bandpass_fmin: 'int' = 0, bandpass_fmax: 'int' = 15000, speed: 'float' = 1.0, apply_sigmoid: 'bool' = True, sigmoid_sensitivity: 'float | None' = 1.0, default_confidence_threshold: 'float | None' = 0.1, custom_confidence_thresholds: 'dict[str, float] | None' = None, custom_species_list: 'str | Path | Collection[str] | None' = None, half_precision: 'bool' = False, max_audio_duration_min: 'float | None' = None, device: 'str | list[str]' = 'CPU', show_stats: "Literal['minimal', 'progress', 'benchmark'] | None" = None, progress_callback: 'Callable[[AcousticProgressStats], None] | None' = None) -> 'AcousticPredictionResultBase'
None


In [ ]:
import time

t0 = time.perf_counter()

predictions = model.predict(
    str(wav_path), 
    overlap_duration_s=2.0,
    default_confidence_threshold=0.1,
    sigmoid_sensitivity=1.0
    )

t1 = time.perf_counter()
print(f"Predict-Zeit: {t1 - t0:.2f} s")

TypeError: AcousticModelV2_4.predict() got an unexpected keyword argument 'locale'

In [12]:
df = predictions.to_dataframe()
df.head(3)

c:\Users\mghan\miniconda3\envs\birdnet_bachelor\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,input,start_time,end_time,species_name,confidence
0,C:\Users\mghan\Desktop\BirdDetection_BA\audio_...,3.0,6.0,Myadestes occidentalis_Brown-backed Solitaire,0.330011
1,C:\Users\mghan\Desktop\BirdDetection_BA\audio_...,4.0,7.0,Myadestes occidentalis_Brown-backed Solitaire,0.362924
2,C:\Users\mghan\Desktop\BirdDetection_BA\audio_...,8.0,11.0,Myadestes occidentalis_Brown-backed Solitaire,0.167974


## Speichern der Datei

In [13]:
zieldatei = OUT_DIR / (str(wav_path.stem) + "_BirdNET.csv" )

AttributeError: 'str' object has no attribute 'stem'

In [21]:
zieldatei = "C:/Users/mghan/OneDrive/01.Learn AI/01.BA_Birdmonitoring/project/outputs/experiments/birdnet/" + str(wav_path.stem) + (".BirdNET.results.csv")

In [22]:
zieldatei

'C:/Users/mghan/OneDrive/01.Learn AI/01.BA_Birdmonitoring/project/outputs/experiments/birdnet/2453AC0263FBD00C_20250610_042001_first30min.BirdNET.results.csv'

In [23]:
predictions.to_csv(zieldatei)

Preparing CSV export...


Writing CSV:   0%|          | 0/512 [00:00<?, ?predictions/s]

Writing CSV: 100%|██████████| 512/512 [00:00<00:00, 34963.91predictions/s, CSV=0 MB]


## Untersuchen der Datei

In [39]:
import pandas as pd

In [42]:
datei_24_04_052500 =  pd.read_csv("../outputs/experiments/birdnet/2453AC0263FBD00C_20250424_052500_BirdNET.csv")

In [47]:
datei_24_04_052500[['input']].iloc[0]

In [2]:
import torch
use_cuda = torch.cuda.is_available()

In [3]:
use_cuda

True

In [4]:
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

__CUDNN VERSION: 91002
__Number CUDA Devices: 1
__CUDA Device Name: Tesla T4
__CUDA Device Total Memory [GB]: 15.828320256


In [5]:
!nvidia-smi


Wed Jan  7 00:13:35 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----